In [ ]:
import os
import logging
import numpy as np
import torch
import json
import random

from config import args

from utils import commonUtils, trainUtils
import dataset
import bertMrc
from preprocess import MRCBertFeature

from torch.utils.data import DataLoader, RandomSampler

from transformers import AdamW
from transformers import BertTokenizer

from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from seqeval.metrics.sequence_labeling import get_entities

In [ ]:
# 配置写入训练记录的logger对象，和全局的随机数种子对象
commonUtils.set_seed(args.seed)
logger = logging.getLogger(__name__)
commonUtils.set_logger(os.path.join(args.log_dir, 'bertMrc.log'))

In [ ]:
# 一定要格外注意这个工具函数
# 其实就是把论元标注转换成了bert模型的阅读理解任务
# 模型接收文本 + 提问，就好像给你阅读理解的原文 + 阅读理解的问题
# 模型返回两个多分类结果
# 一个是start，一个是end
# start中包含对每个词的标注（或者说包含每个词的分类结果），end中也包含对每个词的标注（或者说包含每个词的分类结果）
# 通过输出的start，end，就可以拼装出来BIO格式的标注信息
# 看一下下面的代码，仔细理解一下，其实非常简单
# start其实就是在预测某一具体论元的起始位置，end就是在预测某一具体论元的结束位置
# 他俩在一块就能够确定一个论元
def convert_value_to_bio(start, end, text, id2rolelabel):
    """
    text = '我爱北京的烤鸭'
    labels = {1:'地点',2:'食品'}
    start = [0,0,1,0,0,2,0]
    end = [0,0,0,1,0,0,2]
    convert_value_to_bio(start,end,text,labels)
    ['0', '0', 'B-地点', 'I-地点', '0', 'B-食品', 'I-食品']
    """
    res = ['O'] * len(start)
    length = len(start)
    for i in range(length):
        if start[i] != 0:
            for j in range(i, length):
                if start[i] == end[j]:
                    label = id2rolelabel[start[i]]
                    label = label.replace('-', '_')
                    res[i] = 'B-' + label
                    for k in range(i + 1, j + 1):
                        res[k] = 'I-' + label
                    break
    return res

In [ ]:
# 注意这个不是神经网络类，而是封装了测试、预测逻辑的工具类
# 其实就是把一些工具函数给封装了一下，不把它们封装为工具函数，直接写成顶层函数也是可以的，只是这里遵循这种写法
# 神经网络类是在bertMrc.py中
class BertForMrc:
    def __init__(self, model, train_loader, dev_loader, test_loader, args):
        # 准备一些变量
        self.args = args
        self.model = model
        self.train_loader = train_loader
        self.dev_loader = dev_loader
        self.test_loader = test_loader
        self.device = torch.device(
            'cuda' if torch.cuda.is_available() else 'cpu')
        # self.model, self.device = trainUtils.load_model_and_parallel(
        #     self.model, args.gpu_ids)

        # 计算总训练次数
        if len(self.train_loader) != 0:
            self.t_total = len(self.train_loader) * self.args.train_epochs
        # 这里我们不使用scheduler了，直接使用AdamW优化器
        # self.optimizer, self.scheduler = trainUtils.build_optimizer_and_scheduler(
        #     self.args, self.model, self.t_total)
        self.optimizer = AdamW(self.model.parameters(),
                               lr=args.lr, eps=args.adam_epsilon)

    def train(self, model_name):
        # 训练步数相关
        global_step = 0
        eval_steps = self.t_total // 3  # 全局进行3次eval
        logger.info('每{}个step，会进行验证，全局会进行3次验证'.format(eval_steps), )

        # 初始化最佳F1分数
        best_f1 = 0.0

        # 开始训练
        for epoch in range(self.args.train_epochs):  # epoch
            for step, batch_data in enumerate(self.train_loader):  # batch

                # 他实在是坚持不住，可就算是这样亲人家最终还是看在他面子上，他才同意他把模型切换成为训练模式
                self.model.train()

                # 下面这个for循环就是把下面这些数据移动到device上
                # token_ids
                # attention_masks
                # token_type_ids
                # start_ids
                # end_ids
                for key in batch_data.keys():
                    # print(111, key)
                    batch_data[key] = batch_data[key].to(self.device)

                # 模型预测
                start_logits, end_logits = self.model(batch_data['token_ids'], batch_data['attention_masks'],
                                                      batch_data['token_type_ids'], batch_data['start_ids'],
                                                      batch_data['end_ids'])
                # 计算损失
                loss = self.model.loss(
                    batch_data['start_ids'],
                    batch_data['end_ids'],
                    start_logits,
                    end_logits,
                    batch_data['token_type_ids']
                )

                # 梯度裁剪，避免梯度爆炸
                torch.nn.utils.clip_grad_norm_(
                    self.model.parameters(), self.args.max_grad_norm)
                # loss.backward(loss.clone().detach())
                # print(loss.item())

                # 反向传播
                loss.backward()
                self.optimizer.step()
                # self.scheduler.step()
                self.model.zero_grad()

                # 记录本轮训练日志
                logger.info('[train] epoch:{}/{} step:{}/{} loss:{:.6f}'.format(epoch, self.args.train_epochs,
                                                                                global_step, self.t_total, loss.item()))
                global_step += 1

                # 评估，我们暂时不理会评估代码，也不做模型保存
                # if global_step % eval_steps == 0:
                #     dev_loss, accuracy, precision, recall, f1 = self.dev()
                #     logger.info('[dev] loss:{:.6f} accuracy:{:.4f} precision:{:.4f} recall:{:.4f} f1:{:.4f}'.format(
                #         dev_loss, accuracy, precision, recall, f1))
                #     # 如果评估的f1分数较好的话，则保存模型
                #     if f1 > best_f1:
                #         best_f1 = f1
                #         trainUtils.save_model(
                #             self.args, self.model, model_name, global_step)

    # 我们先不做dev测试
    # def dev(self):
    #     s_logits, e_logits = None, None
    #     true_s_logits, true_e_logits = None, None
    #     self.model.eval()
    #     with torch.no_grad():
    #         for eval_step, dev_batch_data in enumerate(self.dev_loader):
    #             for key in dev_batch_data.keys():
    #                 dev_batch_data[key] = dev_batch_data[key].to(self.device)
    #             batch_size, max_seq_length = dev_batch_data['token_ids'].size()
    #             start_logits, end_logits = self.model(dev_batch_data['token_ids'],
    #                                                   dev_batch_data['attention_masks'],
    #                                                   dev_batch_data['token_type_ids'],
    #                                                   dev_batch_data['start_ids'],
    #                                                   dev_batch_data['end_ids'])
    #             loss = self.model.loss(dev_batch_data['start_ids'], dev_batch_data['end_ids'], start_logits, end_logits,
    #                                    dev_batch_data['token_type_ids'])
    #             start_logits = start_logits.reshape(
    #                 batch_size, max_seq_length, -1).detach().cpu().numpy()
    #             end_logits = end_logits.reshape(
    #                 batch_size, max_seq_length, -1).detach().cpu().numpy()
    #             true_start_ids = dev_batch_data['start_ids'].detach(
    #             ).cpu().numpy()
    #             true_end_ids = dev_batch_data['end_ids'].detach().cpu().numpy()
    #             tmp_start_logits = np.argmax(start_logits, axis=2)
    #             tmp_end_logits = np.argmax(end_logits, axis=2)
    #             if s_logits is None:
    #                 s_logits = tmp_start_logits
    #                 e_logits = tmp_end_logits
    #                 true_s_logits = true_start_ids
    #                 true_e_logits = true_end_ids
    #             else:
    #                 s_logits = np.append(s_logits, tmp_start_logits, axis=0)
    #                 e_logits = np.append(e_logits, tmp_end_logits, axis=0)
    #                 true_s_logits = np.append(
    #                     true_s_logits, true_start_ids, axis=0)
    #                 true_e_logits = np.append(
    #                     true_e_logits, true_end_ids, axis=0)
    #         preds = []
    #         trues = []
    #         for tmp_s_logits, tmp_e_logits, true_tmp_s_logits, true_tmp_e_logits, tmp_callback_info in zip(s_logits,
    #                                                                                                        e_logits,
    #                                                                                                        true_s_logits,
    #                                                                                                        true_e_logits,
    #                                                                                                        dev_callback_info):
    #             text, text_offset, event_type, entities = tmp_callback_info
    #             tmp_s_logits = tmp_s_logits[text_offset:text_offset +
    #                                         len(text)]
    #             tmp_e_logits = tmp_e_logits[text_offset:text_offset +
    #                                         len(text)]
    #             true_tmp_s_logits = true_tmp_s_logits[text_offset:text_offset + len(
    #                 text)]
    #             true_tmp_e_logits = true_tmp_e_logits[text_offset:text_offset + len(
    #                 text)]
    #             pred_bio = convert_value_to_bio(
    #                 tmp_s_logits, tmp_e_logits, text, id2rolelabel)
    #             true_bio = convert_value_to_bio(
    #                 true_tmp_s_logits, true_tmp_e_logits, text, id2rolelabel)
    #             preds.append(pred_bio)
    #             trues.append(true_bio)
    #         accuracy = accuracy_score(trues, preds)
    #         precision = precision_score(trues, preds)
    #         recall = recall_score(trues, preds)
    #         f1 = f1_score(trues, preds)
    #         return loss.item(), accuracy, precision, recall, f1

    # def test(self, model, model_path):
    def test(self, model):
        # model, device = trainUtils.load_model_and_parallel(
        #     model, self.args.gpu_ids, model_path)
        device = torch.device(
            'cuda' if torch.cuda.is_available() else 'cpu')

        s_logits, e_logits = None, None
        true_s_logits, true_e_logits = None, None
        # 进入评估模式
        # 这段代码其实可以参考下面predict的注释，里面我写的非常详细
        model.eval()
        with torch.no_grad():
            for eval_step, test_batch_data in enumerate(self.test_loader):
                for key in test_batch_data.keys():
                    test_batch_data[key] = test_batch_data[key].to(device)
                # 模型的预测输出，并把他们转移到cpu
                start_logits, end_logits = model(test_batch_data['token_ids'], test_batch_data['attention_masks'],
                                                 test_batch_data['token_type_ids'])
                start_logits = start_logits.detach().cpu().numpy()
                end_logits = end_logits.detach().cpu().numpy()
                # 真实输出
                true_start_ids = test_batch_data['start_ids'].detach(
                ).cpu().numpy()
                true_end_ids = test_batch_data['end_ids'].detach(
                ).cpu().numpy()
                # 模型预测输出，转换为索引值
                tmp_start_logits = np.argmax(start_logits, axis=2)
                tmp_end_logits = np.argmax(end_logits, axis=2)
                if s_logits is None:
                    s_logits = tmp_start_logits
                    e_logits = tmp_end_logits
                    true_s_logits = true_start_ids
                    true_e_logits = true_end_ids
                else:
                    s_logits = np.append(s_logits, tmp_start_logits, axis=0)
                    e_logits = np.append(e_logits, tmp_end_logits, axis=0)
                    true_s_logits = np.append(
                        true_s_logits, true_start_ids, axis=0)
                    true_e_logits = np.append(
                        true_e_logits, true_end_ids, axis=0)
            preds = []
            trues = []
            # 计算损失
            for tmp_s_logits, tmp_e_logits, true_tmp_s_logits, true_tmp_e_logits, tmp_callback_info in zip(s_logits,
                                                                                                           e_logits,
                                                                                                           true_s_logits,
                                                                                                           true_e_logits,
                                                                                                           test_callback_info):
                text, text_offset, event_type, entities = tmp_callback_info
                tmp_s_logits = tmp_s_logits[text_offset:text_offset +
                                            len(text)]
                tmp_e_logits = tmp_e_logits[text_offset:text_offset +
                                            len(text)]
                true_tmp_s_logits = true_tmp_s_logits[text_offset:text_offset + len(
                    text)]
                true_tmp_e_logits = true_tmp_e_logits[text_offset:text_offset + len(
                    text)]
                pred_bio = convert_value_to_bio(
                    tmp_s_logits, tmp_e_logits, text, id2rolelabel)
                true_bio = convert_value_to_bio(
                    true_tmp_s_logits, true_tmp_e_logits, text, id2rolelabel)

                preds.append(pred_bio)
                trues.append(true_bio)
            accuracy = accuracy_score(trues, preds)
            precision = precision_score(trues, preds)
            recall = recall_score(trues, preds)
            f1 = f1_score(trues, preds)
            report = classification_report(trues, preds)
            logger.info('[test] accuracy:{} precision:{} recall:{} f1:{}'.format(
                accuracy, precision, recall, f1))
            logger.info(report)

    # 预测函数
    def predict(self, raw_text, query, model, device, args, query2label):
        # print(111, raw_text)
        # 2019年5月10日18时10分，永平县博南镇糖果厂后山突发森林火灾，火情发生后，永平县立即启动森林火灾扑救预案，县委、政府领导靠前指挥，组织扑火力量206人参与扑救。
        # print(222, query)
        # 找出和灾害/意外-起火相关的属性
        # 进入评估模式
        model.to(device)
        model.eval()
        with torch.no_grad():
            tokenizer = BertTokenizer(
                os.path.join(args.bert_dir, 'vocab.txt'))

            # 分别将原始文本和查询转换为token列表
            tokens_b = [i for i in raw_text]
            tokens_a = [i for i in query]
            # print(333, len(tokens_b), tokens_b)
            # 333 84 ['2', '0', '1', '9', '年', '5', '月', '1', '0', '日', '1', '8', '时', '1', '0', '分', '，', '永', '平', '县', '博', '南', '镇', '糖', '果', '厂', '后', '山', '突', '发', '森', '林', '火', '灾', '，', '火', '情', '发', '生', '后', '，', '永', '平', '县', '立', '即', '启', '动', '森', '林', '火', '灾', '扑', '救', '预', '案', '，', '县', '委', '、', '政', '府', '领', '导', '靠', '前', '指', '挥', '，', '组', '织', '扑', '火', '力', '量', '2', '0', '6', '人', '参', '与', '扑', '救', '。']
            # print(444, len(tokens_a), tokens_a)
            # 444 16 ['找', '出', '和', '灾', '害', '/', '意', '外', '-', '起', '火', '相', '关', '的', '属', '性']

            # 把文本和查询配对编码
            encode_dict = tokenizer.encode_plus(text=tokens_a,
                                                text_pair=tokens_b,
                                                max_length=args.max_seq_len,
                                                padding='max_length',
                                                truncation_strategy='only_second',
                                                return_token_type_ids=True,
                                                return_attention_mask=True)

            # print(555, len(encode_dict['input_ids']),
            #       encode_dict['input_ids'])
            # 这里是设置最长接收320个token，其实bert模型一般可以接受512个字符
            # 注意下面有一个102，是[sep]特殊字符
            # 注意这里是先拼接的提问，后拼接的原始文本
            # 555 320 [101, 2823, 1139, 1469, 4135, 2154, 120, 2692, 1912, 118, 6629, 4125, 4685, 1068, 4638, 2247, 2595, 102, 123, 121, 122, 130, 2399, 126, 3299, 122, 121, 3189, 122, 129, 3198, 122, 121, 1146, 8024, 3719, 2398, 1344, 1300, 1298, 7252, 5131, 3362, 1322, 1400, 2255, 4960, 1355, 3481, 3360, 4125, 4135, 8024, 4125, 2658, 1355, 4495, 1400, 8024, 3719, 2398, 1344, 4989, 1315, 1423, 1220, 3481, 3360, 4125, 4135, 2800, 3131, 7564, 3428, 8024, 1344, 1999, 510, 3124, 2424, 7566, 2193, 7479, 1184, 2900, 2916, 8024, 5299, 5302, 2800, 4125, 1213, 7030, 123, 121, 127, 782, 1346, 680, 2800, 3131, 511, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
            # print(555, len(encode_dict['attention_mask']),
            #       encode_dict['attention_mask'])
            # 555 320 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
            # print(555, len(encode_dict['token_type_ids']),
            #       encode_dict['token_type_ids'])
            # 555 320 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
            # 把编码转移到设备上
            token_ids = torch.from_numpy(
                np.array(encode_dict['input_ids'])).unsqueeze(0).to(device)
            attention_masks = torch.from_numpy(
                np.array(encode_dict['attention_mask'])).unsqueeze(0).to(device)
            token_type_ids = torch.from_numpy(
                np.array(encode_dict['token_type_ids'])).unsqueeze(0).to(device)

            # 模型开始预测，得到开始和结束位置的logits
            start_logits, end_logits = model(token_ids,
                                             attention_masks,
                                             token_type_ids)
            # print(666, start_logits.shape, start_logits)
            #  666 torch.Size([1, 320, 218]) tensor([[[ 11.4773, -11.2956, -11.1258,  ..., -11.6017,  -9.4264,  -8.4990],
            # [ 10.1108,  -8.6108, -10.7362,  ...,  -9.6481,  -7.5166,  -6.2054],
            # [ 11.7222,  -9.3814, -11.4222,  ..., -10.5143,  -7.9139,  -6.6573],
            # ...,
            # [  8.3563,  -7.6287,  -7.2876,  ...,  -7.9753,  -5.6169,  -5.1138],
            # [  7.8367,  -7.1326,  -7.1343,  ...,  -7.5717,  -5.3577,  -4.7769],
            # [  8.4908,  -7.5178,  -7.2686,  ...,  -7.9794,  -5.5519,  -5.0563]]])
            # print(777, end_logits.shape, end_logits)
            # 777 torch.Size([1, 320, 218]) tensor([[[ 11.8341, -10.4486, -10.2598,  ...,  -8.9536,  -9.5774,  -9.4688],
            # [ 11.7983,  -8.3412,  -9.1824,  ...,  -7.4878,  -7.9493,  -8.0328],
            # [ 11.0621,  -8.2409,  -9.2021,  ...,  -7.5320,  -8.2500,  -8.7205],
            # ...,
            # [  8.0939,  -6.6059,  -6.1779,  ...,  -5.1880,  -5.9072,  -5.9921],
            # [  8.3923,  -6.5384,  -6.2076,  ...,  -5.1408,  -5.7772,  -5.6470],
            # [  8.3237,  -6.5808,  -6.1862,  ...,  -5.1955,  -5.8844,  -6.0203]]])

            # 将预测结果从GPU移到CPU并转换为numpy数组
            tmp_start_logits = start_logits.detach().cpu().numpy()
            tmp_end_logits = end_logits.detach().cpu().numpy()
            # print(888, tmp_start_logits.shape, tmp_start_logits)
            # 看这里建模成了一个218维的多分类问题其实就是217个论元角色，加上0，一共是218个
            # 888 (1, 320, 218) [[[ 11.477271  -11.29565   -11.125819  ... -11.601685   -9.426403
            #   -8.499003 ]
            # [ 10.110802   -8.610786  -10.736224  ...  -9.648104   -7.5165987
            #   -6.2054296]
            # [ 11.7222     -9.381425  -11.422159  ... -10.514349   -7.913916
            #   -6.657258 ]
            # ...
            # [  8.35628    -7.628654   -7.2875533 ...  -7.9753036  -5.616936
            #   -5.113833 ]
            # [  7.8366523  -7.132631   -7.1342597 ...  -7.571736   -5.3576684
            #   -4.776873 ]
            # [  8.49075    -7.5178337  -7.2686324 ...  -7.9793572  -5.551898
            #   -5.0562553]]]
            # print(999, tmp_end_logits.shape, tmp_end_logits)
            # (1, 320, 218) [[[ 11.83412   -10.448565  -10.259841  ...  -8.953622   -9.577404
            #   -9.468765 ]
            # [ 11.798292   -8.341238   -9.182406  ...  -7.4877567  -7.949266
            #   -8.03279  ]
            # [ 11.062127   -8.240886   -9.202128  ...  -7.5319786  -8.250036
            #   -8.720463 ]
            # ...
            # [  8.093922   -6.6058764  -6.177901  ...  -5.188013   -5.9072313
            #   -5.9921412]
            # [  8.392265   -6.538419   -6.2076335 ...  -5.1408153  -5.777222
            #   -5.647031 ]
            # [  8.323682   -6.5808215  -6.1861734 ...  -5.1954703  -5.8844457
            #   -6.0203457]]]
            text_offset = len(tokens_a) + 2
            tmp_start_logits = np.argmax(tmp_start_logits, axis=2)
            tmp_end_logits = np.argmax(tmp_end_logits, axis=2)
            # print('aaa', text_offset)
            # aaa 18
            # print('bbb', tmp_start_logits.shape, tmp_start_logits)
            # bbb (1, 320) [[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            # 163   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 164
            #   0   0   0   0   0   0   0   0   0   0   0   0 164   0   0   0   0   0
            #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            #   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
            # print('ccc', tmp_end_logits.shape, tmp_end_logits)
            # ccc (1, 320) [[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            # 0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 163   0   0
            # 0   0   0   0   0   0   0   0   0 164   0   0   0 164   0   0   0   0
            # 0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            # 0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            # 0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            # 0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            # 0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            # 0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            # 0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            # 0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            # 0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            # 0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            # 0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            # 0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            # 0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            # 0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
            # 0   0   0   0   0   0   0   0   0   0   0   0   0   0]]

            for t_start_logits, t_end_logits in zip(tmp_start_logits, tmp_end_logits):
                temp_start_logits = t_start_logits[text_offset:text_offset + len(
                    raw_text)]
                temp_end_logits = t_end_logits[text_offset:text_offset +
                                               len(raw_text)]
                # print('ddd', len(temp_start_logits), temp_start_logits)
                # ddd 84 [163   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 164
                # 0   0   0   0   0   0   0   0   0   0   0   0 164   0   0   0   0   0
                # 0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
                # 0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
                # 0   0   0   0   0   0   0   0   0   0   0   0]
                # print('eee', len(temp_end_logits), temp_end_logits)
                # eee 84 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 163   0   0
                # 0   0   0   0   0   0   0   0   0 164   0   0   0 164   0   0   0   0
                # 0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
                # 0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
                # 0   0   0   0   0   0   0   0   0   0   0   0]
                # 将预测的起始和结束位置转换为 BIO 格式的标签
                preds = convert_value_to_bio(
                    temp_start_logits, temp_end_logits, raw_text, id2rolelabel)
                # print('fff', len(preds), preds)
                # fff 84 ['B-灾害/意外_起火_时间', 'I-灾害/意外_起火_时间', 'I-灾害/意外_起火_时间', 'I-灾害/意外_起火_时间', 'I-灾害/意外_起火_时间', 'I-灾害/意外_起火_时间', 'I-灾害/意外_起火_时间', 'I-灾害/意外_起火_时间', 'I-灾害/意外_起火_时间', 'I-灾害/意外_起火_时间', 'I-灾害/意外_起火_时间', 'I-灾害/意外_起火_时间', 'I-灾害/意外_起火_时间', 'I-灾害/意外_起火_时间', 'I-灾害/意外_起火_时间', 'I-灾害/意外_起火_时间', 'O', 'B-灾害/意外_起火_地点', 'I-灾害/意外_起火_地点', 'I-灾害/意外_起火_地点', 'I-灾害/意外_起火_地点', 'I-灾害/意外_起火_地点', 'I-灾害/意外_起火_地点', 'I-灾害/意外_起火_地点', 'I-灾害/意外_起火_地点', 'I-灾害/意外_起火_地点', 'I-灾害/意外_起火_地点', 'I-灾害/意外_起火_地点', 'O', 'O', 'B-灾害/意外_起火_地点', 'I-灾害/意外_起火_地点', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
                preds = get_entities(preds)
                # print('ggg', len(preds), preds)
                # ggg 3 [('灾害/意外_起火_时间', 0, 15), ('灾害/意外_起火_地点', 17, 27), ('灾害/意外_起火_地点', 30, 31)]
                preds = [(pred[0], raw_text[pred[1]:pred[2]+1], pred[1], pred[2])
                         for pred in preds]
                # print('hhh', len(preds), preds)
                # hhh 3 [('灾害/意外_起火_时间', '2019年5月10日18时10分', 0, 15), ('灾害/意外_起火_地点', '永平县博南镇糖果厂后山', 17, 27), ('灾害/意外_起火_地点', '森林', 30, 31)]
                logger.info(preds)


In [ ]:
# 从这里开始真正执行
# 读取训练数据
final_data_path = args.data_dir + 'final_data/'
train_features, train_callback_info = commonUtils.read_pkl(
  final_data_path, 'train')
train_dataset = dataset.MrcDataset(train_features)
train_sampler = RandomSampler(train_dataset)
train_loader = DataLoader(dataset=train_dataset,
                        batch_size=args.train_batch_size,
                        sampler=train_sampler,
                        num_workers=2)

# 读取验证数据，我们先不考虑验证集，来简化一下模型
dev_features, dev_callback_info = commonUtils.read_pkl(
  final_data_path, 'dev')
dev_dataset = dataset.MrcDataset(dev_features)
dev_loader = DataLoader(dataset=dev_dataset,
                      batch_size=args.eval_batch_size,
                      num_workers=2)

# 读取测试数据
test_features, test_callback_info = commonUtils.read_pkl(
  final_data_path, 'test')
test_dataset = dataset.MrcDataset(test_features)
test_loader = DataLoader(dataset=test_dataset,
                        batch_size=args.eval_batch_size,
                        num_workers=2)

# 事件标签
label2id = {}
id2label = {}
with open(final_data_path + 'labels.txt', 'r', encoding='utf-8') as fp:
  labels = fp.read().strip().split('\n')
for i, j in enumerate(labels):
  label2id[j] = i
  id2label[i] = j
# print(111, label2id)
# print(222, id2label)
# 一共是64种事件类型，如果加上None，来表示空事件就是65种
# 111 {'财经/交易-出售/收购': 0, '财经/交易-跌停': 1, '财经/交易-加息': 2, '财经/交易-降价': 3, '财经/交易-降息': 4, '财经/交易-融资': 5, '财经/交易-上市': 6, '财经/交易-涨价': 7, '财经/交易-涨停': 8, '产品行为-发布': 9, '产品行为-获奖': 10, '产品行为-上映': 11, '产品行为-下架': 12, '产品行为-召回': 13, '交往-道歉': 14, '交往-点赞': 15, '交往-感谢': 16, '交往-会见': 17, '交往-探班': 18, '竞赛行为-夺冠': 19, '竞赛行为-晋级': 20, '竞赛行为-禁赛': 21, '竞赛行为-胜负': 22, '竞赛行为-退赛': 23, '竞赛行为-退役': 24, '人生-产子/女': 25, '人生-出轨': 26, '人生-订婚': 27, '人生-分手': 28, '人生-怀 孕': 29, '人生-婚礼': 30, '人生-结婚': 31, '人生-离婚': 32, '人生-庆生': 33, '人生-求婚': 34, '人生-失联': 35, '人生-死亡': 36, '司法行为-罚款': 37, '司法行为-拘捕': 38, '司法行为-举报': 39, '司法行为-开庭': 40, '司法行为-立案': 41, '司法行为-起诉': 42, '司法行为-入狱': 43, '司法行为-约谈': 44, '灾害/意外-爆炸': 45, '灾害/意外-车祸': 46, '灾害/意外-地震': 47, '灾害/意外-洪灾': 48, '灾害/意外-起火': 49, '灾害/意外-坍/垮塌': 50, '灾害/意外-袭击': 51, '灾害/意外-坠机': 52, '组织关系-裁员': 53, '组织关系-辞/离职': 54, '组织关系-加盟': 55, '组织关系-解雇': 56, '组织关系-解散': 57, '组织关系- 解约': 58, '组织关系-停职': 59, '组织关系-退出': 60, '组织行为-罢工': 61, '组织行为-闭幕': 62, '组织行为-开幕': 63, '组织行为-游行': 64}
# 222 {0: '财经/交易-出售/收购', 1: '财经/交易-跌停', 2: '财经/交易-加息', 3: '财经/交易-降价', 4: '财经/交易-降息', 5: '财经/交易-融资', 6: '财经/交易-上市', 7: '财经/交易-涨价', 8: '财经/交易-涨停', 9: '产品行为-发布', 10: '产品行为-获奖', 11: '产品行为-上映', 12: '产品行为-下架', 13: '产品行为- 召回', 14: '交往-道歉', 15: '交往-点赞', 16: '交往-感谢', 17: '交往-会见', 18: '交往-探班', 19: '竞赛行为-夺冠', 20: '竞赛行为-晋级', 21: '竞赛行为-禁赛', 22: '竞赛行为-胜负', 23: '竞赛行为-退赛', 24: '竞赛行为-退役', 25: '人生-产子/女', 26: '人生-出轨', 27: '人生-订婚', 28: '人生-分手', 29: '人 生-怀孕', 30: '人生-婚礼', 31: '人生-结婚', 32: '人生-离婚', 33: '人生-庆生', 34: '人生-求婚', 35: '人生-失联', 36: '人生-死亡', 37: '司法行为-罚款', 38: '司法行为-拘捕', 39: '司法行为-举报', 40: '司法行为-开庭', 41: '司法行为-立案', 42: '司法行为-起诉', 43: '司法行为-入狱', 44: '司法行为-约谈', 45: '灾害/意外-爆炸', 46: '灾害/意外-车祸', 47: '灾害/意外-地震', 48: '灾害/意外-洪灾', 49: '灾害/意外-起火', 50: '灾害/意外-坍/垮塌', 51: '灾害/意外-袭击', 52: '灾害/意外-坠机', 53: '组织关系-裁员', 54: '组织关系-辞/离职', 55: '组织关系-加盟', 56: '组织关系-解雇', 57: '组织关系-解散', 58: '组织 关系-解约', 59: '组织关系-停职', 60: '组织关系-退出', 61: '组织行为-罢工', 62: '组织行为-闭幕', 63: '组织行为-开幕', 64: '组织行为-游行'}

# 读取论元角色标签
rolelabel2id = {}
id2rolelabel = {}
with open(final_data_path + 'rolelabels.txt', 'r', encoding='utf-8') as fp:
  rolelabels = fp.read().strip().split('\n')
# 将0留出来
for i, j in enumerate(rolelabels):
  rolelabel2id[j] = i + 1
  id2rolelabel[i + 1] = j
# print(333, id2rolelabel)
# print(444, id2rolelabel)
# 这里一共217个论元角色，在执行多分类时，后面会再加上一个0，代表这一个字符不属于论元信息，所以将来会建模成一个218维的多分类问题
# 333 {1: '财经/交易-出售/收购-时间', 2: '财经/交易-出售/收购-出售方', 3: '财经/交易-出售/收购-交易物', 4: '财经/交易-出售/收购-出售价格', 5: '财经/交易-出售/收购-收购方', 6: '财经/交易-跌停-时间', 7: '财经/交易-跌停-跌停股票', 8: '财经/交易-加息-时间', 9: '财经/交易-加息-加息幅度', 10: '财经/交易-加息-加息机构', 11: '财经/交易-降价-时间', 12: '财经/交易-降价-降价方', 13: '财经/交易-降价-降价物', 14: '财经/交易-降价-降价幅度', 15: '财经/交易-降息-时间', 16: '财经/交易-降息-降息幅度', 17: '财经/交易-降息-降息机构', 18: '财经/交易-融资-时间', 19: '财经/交易-融资-跟投方', 20: '财经/交易-融资-领投方', 21: '财经/交易-融资-融资轮次', 22: '财经/交易-融资-融资金额', 23: '财经/交易-融资-融资方', 24: '财经/交易-上市-时间', 25: '财经/交易-上市-地点', 26: '财经/交易-上市-上市企业', 27: '财经/交易-上市-融资金额', 28: '财经/交易-涨价-时间', 29: '财经/交易-涨价-涨价幅度', 30: '财经/交易-涨价-涨 价物', 31: '财经/交易-涨价-涨价方', 32: '财经/交易-涨停-时间', 33: '财经/交易-涨停-涨停股票', 34: '产品行为-发布-时间', 35: '产品行为-发布-发布产品', 36: '产品行为-发布-发布方', 37: '产品行为-获奖-时间', 38: '产品行为-获奖-获奖人', 39: '产品行为-获奖-奖项', 40: '产品行为-获奖-颁奖机构', 41: '产品行为-上映-时间', 42: '产品行为-上映-上映方', 43: '产品行为-上映-上映影视', 44: '产品行为-下架-时间', 45: '产品行为-下架-下架产品', 46: '产品行为-下架-被下架方', 47: '产品行为-下架-下架方', 48: '产品行为-召回-时间', 49: '产品行为-召回-召回内容', 50: '产品行为-召回-召回方', 51: '交往-道歉-时间', 52: '交往-道歉-道歉对象', 53: '交往-道歉-道歉者', 54: '交往-点赞-时间', 55: '交往-点赞-点赞方', 56: '交往-点赞-点赞对象', 57: '交往-感谢-时间', 58: '交往-感谢-致谢人', 59: '交往-感谢-被感谢人', 60: '交往-会见-时间', 61: '交往-会见-地点', 62: '交往-会见-会见主体', 63: '交往-会见-会见对象', 64: '交往-探班-时间', 65: '交往-探班-探班主体', 66: '交往-探班-探班对象', 67: '竞赛行为-夺冠-时间', 68: '竞赛行为-夺冠-冠军', 69: '竞赛行为-夺冠-夺冠赛事', 70: '竞赛行为-晋级-时间', 71: '竞赛行为-晋级-晋级方', 72: '竞赛行为-晋级-晋级赛事', 73: '竞赛行为-禁赛-时间', 74: '竞赛行为-禁赛-禁赛时长', 75: '竞赛 行为-禁赛-被禁赛人员', 76: '竞赛行为-禁赛-禁赛机构', 77: '竞赛行为-胜负-时间', 78: '竞赛行为-胜负-败者', 79: '竞赛行为-胜负-胜者', 80: '竞赛行为-胜负-赛事名称', 81: '竞赛行为-退赛-时间', 82: '竞赛行为-退赛-退赛赛事', 83: '竞赛行为-退赛-退赛方', 84: '竞赛行为-退役-时间', 85: '竞赛行为-退役-退役者', 86: '人生-产子/女-时间', 87: '人生-产子/女-产子者', 88: '人生-产子/女-出生者', 89: '人生-出轨-时间', 90: '人生-出轨-出轨方', 91: '人生-出轨-出轨对象', 92: '人生-订婚-时间', 93: '人生-订婚-订婚主体', 94: '人生-分手-时间', 95: '人生-分手-分手双方', 96: '人生-怀孕-时间', 97: '人生-怀孕-怀孕者', 98: '人生-婚礼-时间', 99: '人生-婚礼-地点', 100: '人生-婚礼-参礼人员', 101: '人生-婚礼-结婚双方', 102: '人生-结婚-时间', 103: '人生-结婚-结婚双方', 104: '人生-离婚-时间', 105: '人生-离婚-离婚双方', 106: '人生-庆生-时间', 107: '人生-庆生-生日方', 108: '人生-庆生-生日方年龄', 109: '人生-庆生-庆祝方', 110: '人生-求婚-时间', 111: '人生-求婚-求婚者', 112: '人生-求婚-求婚对象', 113: '人生-失联-时间', 114: '人生-失联-地点', 115: '人生-失联-失联者', 116: '人生-死亡-时间', 117: '人生-死亡-地点', 118: '人生-死亡-死者年龄', 119: '人生-死亡-死者', 120: '司法行为-罚款-时间', 121: '司法行为-罚款-罚款对 象', 122: '司法行为-罚款-执法机构', 123: '司法行为-罚款-罚款金额', 124: '司法行为-拘捕-时间', 125: '司法行为-拘捕-拘捕者', 126: '司法行为-拘捕-被拘捕者', 127: '司法行为-举报-时间', 128: '司法行为-举报-举报发起方', 129: '司法行为-举报-举报对象', 130: '司法行为-开庭-时间', 131: '司法行为-开庭-开庭 法院', 132: '司法行为-开庭-开庭案件', 133: '司法行为-立案-时间', 134: '司法行为-立案-立案机构', 135: '司法行为-立案-立案对象', 136: '司法行为-起诉-时间', 137: '司法行为-起诉-被告', 138: '司法行为-起诉-原告', 139: '司法行为-入狱-时间', 140: '司法行为-入狱-入狱者', 141: '司法行为-入狱-刑期', 142: '司法行为-约谈-时间', 143: '司法行为-约谈-约谈对象', 144: '司法行为-约谈-约谈发起方', 145: '灾害/意外-爆炸-时间', 146: '灾害/意外-爆炸-地点', 147: '灾害/意外-爆炸-死亡人数', 148: '灾害/意外-爆炸-受伤人数', 149: '灾害/意外-车祸-时间', 150: '灾害/意外-车祸-地点', 151: '灾害/意外-车祸-死亡人数', 152: '灾害/意外-车祸-受伤人数', 153: '灾害/意外-地震-时间', 154: '灾害/意外-地震-死亡人数', 155: '灾害/意外-地震-震级', 156: '灾害/意外-地震-震源深度', 157: '灾害/意外-地震-震中', 158: '灾害/意外-地震-受伤人数', 159: '灾害/意外-洪灾-时间', 160: '灾害/意外-洪灾-地点', 161: '灾害/意外-洪灾-死亡人数', 162: '灾害/意外-洪灾-受伤人数', 163: '灾害/意外-起火-时间', 164: '灾害/意外-起火-地点', 165: '灾害/意外-起火-死亡人数', 166: '灾害/意外-起火-受伤人数', 167: '灾害/意外-坍/垮塌-时间', 168: '灾害/意外-坍/垮塌-坍塌主体', 169: '灾害/意外-坍/垮塌-死亡人数', 170: '灾害/意外-坍/垮塌-受伤人数', 171: '灾害/意外-袭击-时间', 172: '灾害/意外-袭击-地点', 173: '灾害/意外-袭击-袭击对象', 174: '灾害/意外-袭击-死亡人数', 175: '灾害/意外-袭击-袭击者', 176: '灾害/意外-袭击-受伤人数', 177: '灾害/意外-坠机-时间', 178: '灾害/意外-坠机-地点', 179: '灾害/意外-坠机-死亡人数', 180: '灾害/意外-坠机-受伤人数', 181: '组织关系-裁员-时间', 182: '组织关系-裁员-裁员方', 183: '组织关系-裁员-裁员人数', 184: '组织关系-辞/离职-时间', 185: '组织关系-辞/离职-离职者', 186: '组织关系-辞/离职-原所属组织', 187: '组织关系-加盟-时间', 188: '组织关系-加盟-加盟者', 189: '组织关系-加盟-所加盟组织', 190: '组织关系-解雇-时间', 191: '组织关系-解雇-解雇方', 192: '组织关系-解雇-被解雇人员', 193: '组织关系-解散-时间', 194: '组织关系-解散-解散方', 195: '组织关系-解约-时间', 196: '组织关系-解约-被解约方', 197: '组织关系-解约-解约方', 198: '组织关系-停职-时间', 199: '组织关系-停职-所属组织', 200: '组织关系-停职-停职人员', 201: '组织关系-退出-时间', 202: '组织关系-退出-退出方', 203: '组织关系-退出-原所属组织', 204: '组织行为-罢工-时间', 205: '组织行为-罢工-所属组织', 206: '组织行为-罢工-罢工人数', 207: '组织行为-罢工-罢工人员', 208: '组织行为-闭幕-时间', 209: '组织行为-闭幕-地点', 210: '组织行为-闭幕-活动名称', 211: ' 组织行为-开幕-时间', 212: '组织行为-开幕-地点', 213: '组织行为-开幕-活动名称', 214: '组织行为-游行-时间', 215: '组织行为-游行-地点', 216: '组织行为-游行-游行组织', 217: '组织行为-游行-游行人数'}

# 444 {1: '财经/交易-出售/收购-时间', 2: '财经/交易-出售/收购-出售方', 3: '财经/交易-出售/收购-交易物', 4: '财经/交易-出售/收购-出售价格', 5: '财经/交易-出售/收购-收购方', 6: '财经/交易-跌停-时间', 7: '财经/交易-跌停-跌停股票', 8: '财经/交易-加息-时间', 9: '财经/交易-加息-加息幅度', 10: '财经/交易-加息-加息机构', 11: '财经/交易-降价-时间', 12: '财经/交易-降价-降价方', 13: '财经/交易-降价-降价物', 14: '财经/交易-降价-降价幅度', 15: '财经/交易-降息-时间', 16: '财经/交易-降息-降息幅度', 17: '财经/交易-降息-降息机构', 18: '财经/交易-融资-时间', 19: '财经/交易-融资-跟投方', 20: '财经/交易-融资-领投方', 21: '财经/交易-融资-融资轮次', 22: '财经/交易-融资-融资金额', 23: '财经/交易-融资-融资方', 24: '财经/交易-上市-时间', 25: '财经/交易-上市-地点', 26: '财经/交易-上市-上市企业', 27: '财经/交易-上市-融资金额', 28: '财经/交易-涨价-时间', 29: '财经/交易-涨价-涨价幅度', 30: '财经/交易-涨价-涨 价物', 31: '财经/交易-涨价-涨价方', 32: '财经/交易-涨停-时间', 33: '财经/交易-涨停-涨停股票', 34: '产品行为-发布-时间', 35: '产品行为-发布-发布产品', 36: '产品行为-发布-发布方', 37: '产品行为-获奖-时间', 38: '产品行为-获奖-获奖人', 39: '产品行为-获奖-奖项', 40: '产品行为-获奖-颁奖机构', 41: '产品行为-上映-时间', 42: '产品行为-上映-上映方', 43: '产品行为-上映-上映影视', 44: '产品行为-下架-时间', 45: '产品行为-下架-下架产品', 46: '产品行为-下架-被下架方', 47: '产品行为-下架-下架方', 48: '产品行为-召回-时间', 49: '产品行为-召回-召回内容', 50: '产品行为-召回-召回方', 51: '交往-道歉-时间', 52: '交往-道歉-道歉对象', 53: '交往-道歉-道歉者', 54: '交往-点赞-时间', 55: '交往-点赞-点赞方', 56: '交往-点赞-点赞对象', 57: '交往-感谢-时间', 58: '交往-感谢-致谢人', 59: '交往-感谢-被感谢人', 60: '交往-会见-时间', 61: '交往-会见-地点', 62: '交往-会见-会见主体', 63: '交往-会见-会见对象', 64: '交往-探班-时间', 65: '交往-探班-探班主体', 66: '交往-探班-探班对象', 67: '竞赛行为-夺冠-时间', 68: '竞赛行为-夺冠-冠军', 69: '竞赛行为-夺冠-夺冠赛事', 70: '竞赛行为-晋级-时间', 71: '竞赛行为-晋级-晋级方', 72: '竞赛行为-晋级-晋级赛事', 73: '竞赛行为-禁赛-时间', 74: '竞赛行为-禁赛-禁赛时长', 75: '竞赛 行为-禁赛-被禁赛人员', 76: '竞赛行为-禁赛-禁赛机构', 77: '竞赛行为-胜负-时间', 78: '竞赛行为-胜负-败者', 79: '竞赛行为-胜负-胜者', 80: '竞赛行为-胜负-赛事名称', 81: '竞赛行为-退赛-时间', 82: '竞赛行为-退赛-退赛赛事', 83: '竞赛行为-退赛-退赛方', 84: '竞赛行为-退役-时间', 85: '竞赛行为-退役-退役者', 86: '人生-产子/女-时间', 87: '人生-产子/女-产子者', 88: '人生-产子/女-出生者', 89: '人生-出轨-时间', 90: '人生-出轨-出轨方', 91: '人生-出轨-出轨对象', 92: '人生-订婚-时间', 93: '人生-订婚-订婚主体', 94: '人生-分手-时间', 95: '人生-分手-分手双方', 96: '人生-怀孕-时间', 97: '人生-怀孕-怀孕者', 98: '人生-婚礼-时间', 99: '人生-婚礼-地点', 100: '人生-婚礼-参礼人员', 101: '人生-婚礼-结婚双方', 102: '人生-结婚-时间', 103: '人生-结婚-结婚双方', 104: '人生-离婚-时间', 105: '人生-离婚-离婚双方', 106: '人生-庆生-时间', 107: '人生-庆生-生日方', 108: '人生-庆生-生日方年龄', 109: '人生-庆生-庆祝方', 110: '人生-求婚-时间', 111: '人生-求婚-求婚者', 112: '人生-求婚-求婚对象', 113: '人生-失联-时间', 114: '人生-失联-地点', 115: '人生-失联-失联者', 116: '人生-死亡-时间', 117: '人生-死亡-地点', 118: '人生-死亡-死者年龄', 119: '人生-死亡-死者', 120: '司法行为-罚款-时间', 121: '司法行为-罚款-罚款对 象', 122: '司法行为-罚款-执法机构', 123: '司法行为-罚款-罚款金额', 124: '司法行为-拘捕-时间', 125: '司法行为-拘捕-拘捕者', 126: '司法行为-拘捕-被拘捕者', 127: '司法行为-举报-时间', 128: '司法行为-举报-举报发起方', 129: '司法行为-举报-举报对象', 130: '司法行为-开庭-时间', 131: '司法行为-开庭-开庭 法院', 132: '司法行为-开庭-开庭案件', 133: '司法行为-立案-时间', 134: '司法行为-立案-立案机构', 135: '司法行为-立案-立案对象', 136: '司法行为-起诉-时间', 137: '司法行为-起诉-被告', 138: '司法行为-起诉-原告', 139: '司法行为-入狱-时间', 140: '司法行为-入狱-入狱者', 141: '司法行为-入狱-刑期', 142: '司法行为-约谈-时间', 143: '司法行为-约谈-约谈对象', 144: '司法行为-约谈-约谈发起方', 145: '灾害/意外-爆炸-时间', 146: '灾害/意外-爆炸-地点', 147: '灾害/意外-爆炸-死亡人数', 148: '灾害/意外-爆炸-受伤人数', 149: '灾害/意外-车祸-时间', 150: '灾害/意外-车祸-地点', 151: '灾害/意外-车祸-死亡人数', 152: '灾害/意外-车祸-受伤人数', 153: '灾害/意外-地震-时间', 154: '灾害/意外-地震-死亡人数', 155: '灾害/意外-地震-震级', 156: '灾害/意外-地震-震源深度', 157: '灾害/意外-地震-震中', 158: '灾害/意外-地震-受伤人数', 159: '灾害/意外-洪灾-时间', 160: '灾害/意外-洪灾-地点', 161: '灾害/意外-洪灾-死亡人数', 162: '灾害/意外-洪灾-受伤人数', 163: '灾害/意外-起火-时间', 164: '灾害/意外-起火-地点', 165: '灾害/意外-起火-死亡人数', 166: '灾害/意外-起火-受伤人数', 167: '灾害/意外-坍/垮塌-时间', 168: '灾害/意外-坍/垮塌-坍塌主体', 169: '灾害/意外-坍/垮塌-死亡人数', 170: '灾害/意外-坍/垮塌-受伤人数', 171: '灾害/意外-袭击-时间', 172: '灾害/意外-袭击-地点', 173: '灾害/意外-袭击-袭击对象', 174: '灾害/意外-袭击-死亡人数', 175: '灾害/意外-袭击-袭击者', 176: '灾害/意外-袭击-受伤人数', 177: '灾害/意外-坠机-时间', 178: '灾害/意外-坠机-地点', 179: '灾害/意外-坠机-死亡人数', 180: '灾害/意外-坠机-受伤人数', 181: '组织关系-裁员-时间', 182: '组织关系-裁员-裁员方', 183: '组织关系-裁员-裁员人数', 184: '组织关系-辞/离职-时间', 185: '组织关系-辞/离职-离职者', 186: '组织关系-辞/离职-原所属组织', 187: '组织关系-加盟-时间', 188: '组织关系-加盟-加盟者', 189: '组织关系-加盟-所加盟组织', 190: '组织关系-解雇-时间', 191: '组织关系-解雇-解雇方', 192: '组织关系-解雇-被解雇人员', 193: '组织关系-解散-时间', 194: '组织关系-解散-解散方', 195: '组织关系-解约-时间', 196: '组织关系-解约-被解约方', 197: '组织关系-解约-解约方', 198: '组织关系-停职-时间', 199: '组织关系-停职-所属组织', 200: '组织关系-停职-停职人员', 201: '组织关系-退出-时间', 202: '组织关系-退出-退出方', 203: '组织关系-退出-原所属组织', 204: '组织行为-罢工-时间', 205: '组织行为-罢工-所属组织', 206: '组织行为-罢工-罢工人数', 207: '组织行为-罢工-罢工人员', 208: '组织行为-闭幕-时间', 209: '组织行为-闭幕-地点', 210: '组织行为-闭幕-活动名称', 211: ' 组织行为-开幕-时间', 212: '组织行为-开幕-地点', 213: '组织行为-开幕-活动名称', 214: '组织行为-游行-时间', 215: '组织行为-游行-地点', 216: '组织行为-游行-游行组织', 217: '组织行为-游行-游行人数'}

# 所有论元角色
role_labels = rolelabel2id.keys()
# print(555, role_labels)
# 555 dict_keys(['财经/交易-出售/收购-时间', '财经/交易-出售/收购-出售方', '财经/交易-出售/收购-交易物', '财经/交易-出售/收购-出售价格', '财经/交易-出售/收购-收购方', '财经/交易-跌停-时间', '财经/交易-跌停-跌停股票', '财经/交易-加息-时间', '财经/交易-加息-加息幅度', '财经/交易-加息-加息机构', '财经/交易-降价-时间', '财经/交易-降价-降价方', '财经/交易-降价-降价物', '财经/交易-降价-降价幅度', '财经/交易-降息-时间', '财经/交易-降息-降息幅度', '财经/交易-降息-降息机构', '财经/交易-融资-时间', '财经/交易-融资-跟投方', '财经/交易-融资-领投方', '财经/交易-融资-融资轮次', '财经/交易-融资-融资金额', '财经/交易-融资-融资方', '财经/交易-上市-时间', '财经/交易-上市-地点', '财经/交易-上市-上市企业', '财经/交易-上市-融资金额', '财经/交易-涨价-时间', '财经/交易-涨价-涨价幅度', '财经/交易-涨价-涨价物', '财经/交易-涨价-涨价方', '财经/交易-涨停-时间', '财经/交易-涨停-涨停股票', '产品行为-发布-时间', '产品行为-发布-发布产品', '产品行为-发布-发布方', '产品行为-获奖-时间', '产品行为-获奖-获奖人', '产品行为-获奖-奖项', '产品行为-获奖-颁奖机构', '产品行为-上映-时间', '产品行为-上映-上映方', '产品行为-上映-上映影视', '产品行为-下架-时间', '产品行为-下架-下架产品', '产品行为-下架-被下架方', '产品行为-下架-下架方', '产品行为-召回-时间', '产品行为-召回-召回内容', '产品行为-召回-召回方', '交往-道歉-时间', '交往-道歉-道歉对象', '交往-道歉-道歉者', '交往-点赞-时间', '交往-点赞-点赞方', '交往-点赞-点赞对象', '交往-感谢-时间', '交往-感谢-致谢人', '交往-感谢-被感谢人', '交往-会见-时间', '交往-会见-地点', '交往-会见-会见主体', '交往-会见-会见对象', '交往-探班-时间', '交往-探班-探班主体', '交往-探班-探班对象', '竞赛行为-夺冠-时间', '竞赛行为-夺冠-冠军', '竞赛行为-夺冠-夺冠赛事', '竞赛行为-晋级-时间', '竞赛行为-晋级-晋级方', '竞赛行为-晋级-晋级赛事', '竞赛行为-禁赛-时间', '竞赛行为-禁赛-禁赛时长', '竞赛行为-禁赛-被禁赛人员', '竞赛行为-禁赛-禁赛机构', '竞赛行为-胜负-时间', '竞赛行为-胜负-败者', '竞赛行为-胜负-胜者', '竞赛行为-胜负-赛事名称', '竞赛行为-退赛-时间', '竞赛行为-退赛-退赛赛事', '竞赛行为-退赛-退赛方', '竞赛行为-退役-时间', '竞赛行为-退役-退役者', '人生-产子/女-时间', '人生-产子/女-产子者', '人生-产子/女-出生者', '人生-出轨-时间', '人生-出轨-出轨方', '人生-出轨-出轨对象', '人生-订婚-时间', '人生-订婚-订婚主体', '人生-分手-时间', '人生-分手-分手双方', '人生-怀孕-时间', '人生-怀孕-怀孕者', '人生-婚礼-时间', '人生-婚礼-地点', '人生-婚礼-参礼人员', '人生-婚礼-结婚双 方', '人生-结婚-时间', '人生-结婚-结婚双方', '人生-离婚-时间', '人生-离婚-离婚双方', '人生-庆生-时间', '人生-庆生-生日方', '人生-庆生-生日方年龄', '人生-庆生-庆祝方', '人生-求婚-时间', '人生-求婚-求婚者', '人生-求婚-求婚对象', '人生-失联-时间', '人生-失联-地点', '人生-失联-失联者', '人生-死亡-时 间', '人生-死亡-地点', '人生-死亡-死者年龄', '人生-死亡-死者', '司法行为-罚款-时间', '司法行为-罚款-罚款对象', '司法行为-罚款-执法机构', '司法行为-罚款-罚款金额', '司法行为-拘捕-时间', '司法行为-拘捕-拘捕者', '司法行为-拘捕-被拘捕者', '司法行为-举报-时间', '司法行为-举报-举报发起方', '司法行为-举报-举报对象', '司法行为-开庭-时间', '司法行为-开庭-开庭法院', '司法行为-开庭-开庭案件', '司法行为-立案-时间', '司法行为-立案-立案机构', '司法行为-立案-立案对象', '司法行为-起诉-时间', '司法行为-起诉-被告', '司法行为-起诉-原告', '司法行为-入狱-时间', '司法行为-入狱-入狱者', '司法行为-入狱-刑期', '司法行为-约谈-时间', '司法行为-约谈-约谈对象', '司法行为-约谈-约谈发起方', '灾害/意外-爆炸-时间', '灾害/意外-爆炸-地点', '灾害/意外-爆炸-死亡人数', '灾害/意外-爆炸-受伤人数', '灾害/意外-车祸-时间', '灾害/意外-车祸-地点', '灾害/意外-车祸-死亡人数', '灾害/意外-车祸-受伤人数', '灾害/意外-地震-时间', '灾害/意外-地震-死亡人数', '灾害/意外-地震-震级', '灾害/意外-地震-震源深度', '灾害/意外-地震-震中', '灾害/意外-地震-受伤人数', '灾害/意外-洪灾-时间', '灾害/意外-洪灾-地点', '灾害/意外-洪灾-死亡人数', '灾害/意外-洪灾-受伤人数', '灾害/意外-起火-时间', '灾害/意外-起火-地点', '灾害/意外-起火-死亡人 数', '灾害/意外-起火-受伤人数', '灾害/意外-坍/垮塌-时间', '灾害/意外-坍/垮塌-坍塌主体', '灾害/意外-坍/垮塌-死亡人数', '灾害/意外-坍/垮塌-受伤人数', '灾害/意外-袭击-时间', '灾害/意外-袭击-地点', '灾害/意外-袭击-袭击对象', '灾害/意外-袭击-死亡人数', '灾害/意外-袭击-袭击者', '灾害/意外-袭击-受伤人数', '灾害/意外-坠机-时间', '灾害/意外-坠机-地点', '灾害/意外-坠机-死亡人数', '灾害/意外-坠机-受伤人数', '组织关系-裁员-时间', '组织关系-裁员-裁员方', '组织关系-裁员-裁员人数', '组织关系-辞/离职-时间', '组织关系-辞/离职-离职者', '组织关系-辞/离职-原所属组织', '组织关系-加盟-时间', '组织关系-加盟-加盟者', '组织关系-加盟-所加盟组织', '组织关系-解雇-时间', '组织关系-解雇-解雇方', '组织关系-解雇-被解雇人员', '组织关系-解散-时间', '组织关系-解散-解散方', '组织关系-解约-时间', '组织关系-解约-被解约方', '组织关系-解约-解约方', '组织关系-停职-时间', '组织关系-停职-所属组织', '组织关系-停职-停职人 员', '组织关系-退出-时间', '组织关系-退出-退出方', '组织关系-退出-原所属组织', '组织行为-罢工-时间', '组织行为-罢工-所属组织', '组织行为-罢工-罢工人数', '组织行为-罢工-罢工人员', '组织行为-闭幕-时间', '组织行为-闭幕-地点', '组织行为-闭幕-活动名称', '组织行为-开幕-时间', '组织行为-开幕-地点', '组 织行为-开幕-活动名称', '组织行为-游行-时间', '组织行为-游行-地点', '组织行为-游行-游行组织', '组织行为-游行-游行人数'])

# 读取事件到问答的映射，其实就相当于是阅读理解中的问题
with open(final_data_path + 'labels2query.json', 'r', encoding='utf-8') as fp:
  data = fp.read()
  labels2query = eval(data)
# 事件到论元的映射
with open(final_data_path + 'labels2rolelabels.json', 'r', encoding='utf-8') as fp:
  labels2rolelabels = json.loads(fp.read())

# 创建模型
model = bertMrc.BertMrcModel(args.bert_dir, args)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
# 这是模型的结构
# print(model)
# BertMrcModel(
#   (bert_module): BertModel(
#     (embeddings): BertEmbeddings(
#       (word_embeddings): Embedding(21128, 768, padding_idx=0)
#       (position_embeddings): Embedding(512, 768)
#       (token_type_embeddings): Embedding(2, 768)
#       (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
#       (dropout): Dropout(p=0.1, inplace=False)
#     )
#     (encoder): BertEncoder(
#       (layer): ModuleList(
#         (0-11): 12 x BertLayer(
#           (attention): BertAttention(
#             (self): BertSelfAttention(
#               (query): Linear(in_features=768, out_features=768, bias=True)
#               (key): Linear(in_features=768, out_features=768, bias=True)
#               (value): Linear(in_features=768, out_features=768, bias=True)
#               (dropout): Dropout(p=0.1, inplace=False)
#             )
#             (output): BertSelfOutput(
#               (dense): Linear(in_features=768, out_features=768, bias=True)
#               (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
#               (dropout): Dropout(p=0.1, inplace=False)
#             )
#           )
#           (intermediate): BertIntermediate(
#             (dense): Linear(in_features=768, out_features=3072, bias=True)
#             (intermediate_act_fn): GELUActivation()
#           )
#           (output): BertOutput(
#             (dense): Linear(in_features=3072, out_features=768, bias=True)
#             (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
#             (dropout): Dropout(p=0.1, inplace=False)
#           )
#         )
#       )
#     )
#     (pooler): BertPooler(
#       (dense): Linear(in_features=768, out_features=768, bias=True)
#       (activation): Tanh()
#     )
#   )
#   (mid_linear): Sequential(
#     (0): Linear(in_features=768, out_features=128, bias=True)
#     (1): ReLU()
#     (2): Dropout(p=0.3, inplace=False)
#   )
#   (start_fc): Linear(in_features=128, out_features=218, bias=True)
#   (end_fc): Linear(in_features=128, out_features=218, bias=True)
#   (criterion): CrossEntropyLoss()
# )

In [ ]:
# 准备开始执行训练，预测之类的任务
bertForMrc = BertForMrc(model, train_loader, dev_loader, test_loader, args)

In [ ]:
# ① 训练任务，如果注释掉的话，则代表不进行训练，直接进行预测
bertForMrc.train('bertMrc')

# ① 或者不训练模型，直接从本地加载一个测试用的训练好的参数，但是这个模型太大了，暂时没有放在github上面
# model.load_state_dict(torch.load('./checkpoints/for_test/model.pt', map_location=device))

In [ ]:
# ② 评估任务，会给出F1得分
bertForMrc.test(model)

In [ ]:
# ③ 预测任务1
# 这里是从dev.json中读取10条数据数据进行预测
with open(args.data_dir + 'raw_data/' + 'dev.json', 'r', encoding='utf-8') as fp:
    test_data = fp.readlines()
    test_data = random.sample(test_data, 10)
    for i, line in enumerate(test_data):
        raw_dict = eval(line)
        text = raw_dict['text']
        event_list = raw_dict['event_list']

        for event in event_list:
            # 在预测时需要提前知道事件的类型
            event_type = event['event_type']
            # 然后根据事件类型拿到对应的提问语句
            query = labels2query[event_type]

            logger.info("==============================================")
            logger.info("文本：")
            logger.info(text)
            logger.info("预测值：")
            bertForMrc.predict(
                text,
                query,
                model,
                device,
                args,
                id2rolelabel
            )
# 下面是一个预测实例
# 文本：
# 2019年5月10日18时10分，永平县博南镇糖果厂后山突发森林火灾，火情发生后，永平县立即启动森林火灾扑救预案，县委、政府领导靠前指挥，组织扑火力量206人参与扑救。
# 预测值：
# [('灾害/意外_起火_时间', '2019年5月10日18时10分', 0, 15), ('灾害/意外_起火_地点', '永平县博南镇糖果厂后山', 17, 27), ('灾害/意外_起火_地点', '森林', 30, 31)]

In [ ]:
# ③ 预测任务2，这里也可以传入我们自己的数据，尝试进行预测
# 分别传入的是原始文本和问题，让模型做阅读理解
bertForMrc.predict(
  "2019年，习近平主席热情称赞了国家图书馆工作人员的辛勤付出", # test
  "找出和交往-点赞相关的属性", # query
  model,
  device,
  args,
  id2rolelabel
)
# 这里识别出来它是一个“交往-点赞”事件
# 同时识别出来了两个论元，分别是点赞方和点赞对象
# [('交往_点赞_点赞方', '习近平主席', 6, 10), ('交往_点赞_点赞对象', '国家图书馆工作人员', 16, 24)]